# Customer Segmentation Using K-Means Clustering

## Project Overview
This notebook implements customer segmentation using unsupervised learning (K-Means clustering) on the Online Retail II dataset. The goal is to identify distinct customer groups based on their purchasing behavior using RFM (Recency, Frequency, Monetary) analysis.

### Key Objectives:
- Perform data preprocessing and cleaning
- Engineer RFM features for customer behavior analysis
- Apply K-Means clustering to segment customers
- Evaluate clustering quality using the Elbow Method and Silhouette Score
- Provide business insights for targeted marketing strategies

---
**Author:** Data Science Portfolio Project  
**Dataset:** [Online Retail Dataset](https://www.kaggle.com/datasets/minalchoudhary/online-retail-dataset)  
**License:** MIT

## 1. Setup and Data Loading

First, we import the necessary libraries and load the dataset.

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import warnings

warnings.filterwarnings('ignore')

# Set visualization style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')

print("Libraries imported successfully!")

In [ ]:
# Option 1: Load dataset using kagglehub (if running on Colab/Kaggle)
# import kagglehub
# from kagglehub import KaggleDatasetAdapter
# df = kagglehub.load_dataset(
#     KaggleDatasetAdapter.PANDAS,
#     "minalchoudhary/online-retail-dataset",
#     "online_retail_II.xlsx.csv",
#     pandas_kwargs={"encoding": "latin1"}
# )

# Option 2: Load dataset from local file
# df = pd.read_csv('data/online_retail_II.csv', encoding='latin1')

# For demonstration, using kagglehub
import kagglehub
from kagglehub import KaggleDatasetAdapter

df = kagglehub.load_dataset(
    KaggleDatasetAdapter.PANDAS,
    "minalchoudhary/online-retail-dataset",
    "online_retail_II.xlsx.csv",
    pandas_kwargs={"encoding": "latin1"}
)

# Display basic information about the dataset
print("Dataset Shape:", df.shape)
print("\nFirst 5 rows:")
df.head()

In [ ]:
# Examine dataset structure
print("Dataset Information:")
print("=" * 50)
df.info()
print("\nStatistical Summary:")
df.describe()

## 2. Data Preprocessing

Data cleaning is essential for accurate clustering. We will:
1. Remove records with missing Customer IDs
2. Filter out negative quantities and prices (returns/cancellations)
3. Convert date columns to proper datetime format
4. Calculate total transaction value

In [ ]:
# Check for missing values
print("Missing Values Before Cleaning:")
print(df.isnull().sum())
print(f"\nTotal rows: {len(df)}")

In [ ]:
# Step 1: Remove records with missing Customer IDs
# Customer ID is essential for segmentation
df = df.dropna(subset=["Customer ID"])
print(f"Rows after removing null Customer IDs: {len(df)}")

In [ ]:
# Step 2: Filter out returns and invalid transactions
# Negative quantities indicate returns, negative prices are data errors
df = df[(df["Quantity"] > 0) & (df["Price"] > 0)]
print(f"Rows after removing invalid transactions: {len(df)}")

In [ ]:
# Step 3: Convert InvoiceDate to datetime
df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"], format="%d-%m-%Y %H:%M")
print(f"Date range: {df['InvoiceDate'].min()} to {df['InvoiceDate'].max()}")

In [ ]:
# Step 4: Calculate Total Price for each transaction
df["TotalPrice"] = df["Quantity"] * df["Price"]

# Verify preprocessing results
print("\nCleaned Dataset Preview:")
print(df.head())
print(f"\nFinal dataset shape: {df.shape}")

## 3. RFM Feature Engineering

RFM analysis is a proven marketing technique that evaluates customers based on:

- **Recency (R):** Days since last purchase - lower is better
- **Frequency (F):** Number of unique transactions - higher is better
- **Monetary (M):** Total amount spent - higher is better

These features capture essential aspects of customer purchasing behavior.

In [ ]:
# Set reference date (one day after the last transaction)
reference_date = df["InvoiceDate"].max() + pd.Timedelta(days=1)
print(f"Reference date for Recency calculation: {reference_date}")

# Calculate RFM metrics for each customer
rfm = df.groupby("Customer ID").agg({
    "InvoiceDate": lambda x: (reference_date - x.max()).days,  # Recency
    "Invoice": "nunique",                                       # Frequency
    "TotalPrice": "sum"                                         # Monetary
})

# Rename columns for clarity
rfm.columns = ["Recency", "Frequency", "Monetary"]

print("\nRFM Table Preview:")
print(rfm.head(10))
print(f"\nNumber of unique customers: {len(rfm)}")

In [ ]:
# RFM Statistical Summary
print("RFM Statistical Summary:")
print("=" * 50)
rfm.describe()

In [ ]:
# Visualize RFM distributions
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Recency distribution
axes[0].hist(rfm['Recency'], bins=50, color='steelblue', edgecolor='white')
axes[0].set_title('Recency Distribution', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Days Since Last Purchase')
axes[0].set_ylabel('Number of Customers')

# Frequency distribution
axes[1].hist(rfm['Frequency'], bins=50, color='darkorange', edgecolor='white')
axes[1].set_title('Frequency Distribution', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Number of Transactions')
axes[1].set_ylabel('Number of Customers')

# Monetary distribution
axes[2].hist(rfm['Monetary'], bins=50, color='forestgreen', edgecolor='white')
axes[2].set_title('Monetary Distribution', fontsize=12, fontweight='bold')
axes[2].set_xlabel('Total Spend ($)')
axes[2].set_ylabel('Number of Customers')

plt.tight_layout()
plt.savefig('../images/rfm_distributions.png', dpi=150, bbox_inches='tight')
plt.show()

## 4. Feature Scaling

K-Means is distance-based, making it sensitive to feature scales. StandardScaler normalizes features to have zero mean and unit variance, ensuring equal contribution from each RFM dimension.

In [ ]:
# Apply StandardScaler to normalize RFM features
scaler = StandardScaler()
rfm_scaled = scaler.fit_transform(rfm)

# Convert to DataFrame for easier inspection
rfm_scaled_df = pd.DataFrame(rfm_scaled, 
                              columns=['Recency_Scaled', 'Frequency_Scaled', 'Monetary_Scaled'],
                              index=rfm.index)

print("Scaled RFM Features Preview:")
print(rfm_scaled_df.head())
print("\nScaled Features Statistics:")
print(rfm_scaled_df.describe())

## 5. Determining Optimal Number of Clusters

We use two methods to find the optimal number of clusters:

1. **Elbow Method:** Plot WCSS (Within-Cluster Sum of Squares) vs. number of clusters. The "elbow" point indicates optimal k.

2. **Silhouette Score:** Measures how similar objects are to their own cluster compared to other clusters. Higher is better.

In [ ]:
# Elbow Method - Calculate WCSS for different k values
wcss = []
silhouette_scores = []
K_range = range(2, 11)

for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(rfm_scaled)
    wcss.append(kmeans.inertia_)
    silhouette_scores.append(silhouette_score(rfm_scaled, kmeans.labels_))

print("Cluster Evaluation Metrics:")
print("=" * 40)
for k, w, s in zip(K_range, wcss, silhouette_scores):
    print(f"k={k}: WCSS={w:.2f}, Silhouette={s:.4f}")

In [ ]:
# Visualize Elbow Method and Silhouette Scores
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Elbow Plot
axes[0].plot(K_range, wcss, 'bo-', linewidth=2, markersize=8)
axes[0].axvline(x=4, color='red', linestyle='--', label='Optimal k=4')
axes[0].set_xlabel('Number of Clusters (k)', fontsize=11)
axes[0].set_ylabel('WCSS (Within-Cluster Sum of Squares)', fontsize=11)
axes[0].set_title('Elbow Method for Optimal k', fontsize=13, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Silhouette Score Plot
axes[1].plot(K_range, silhouette_scores, 'go-', linewidth=2, markersize=8)
axes[1].axvline(x=4, color='red', linestyle='--', label='Optimal k=4')
axes[1].set_xlabel('Number of Clusters (k)', fontsize=11)
axes[1].set_ylabel('Silhouette Score', fontsize=11)
axes[1].set_title('Silhouette Score Analysis', fontsize=13, fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../images/cluster_evaluation.png', dpi=150, bbox_inches='tight')
plt.show()

## 6. K-Means Clustering (k=4)

Based on the Elbow Method and Silhouette Score analysis, we select **k=4** as the optimal number of clusters.

In [ ]:
# Train final K-Means model with k=4
kmeans = KMeans(n_clusters=4, random_state=42, n_init=10)
rfm["Cluster"] = kmeans.fit_predict(rfm_scaled)

# Display cluster assignments
print("Cluster Assignment Preview:")
print(rfm.head(10))
print(f"\nCluster Distribution:")
print(rfm['Cluster'].value_counts().sort_index())

In [ ]:
# Calculate final Silhouette Score
final_silhouette = silhouette_score(rfm_scaled, rfm['Cluster'])
print(f"\nFinal Model Silhouette Score: {final_silhouette:.4f}")

## 7. Cluster Analysis and Business Insights

Let's analyze the characteristics of each customer segment to derive actionable business insights.

In [ ]:
# Calculate cluster statistics
cluster_summary = rfm.groupby('Cluster').agg({
    'Recency': ['mean', 'median', 'min', 'max'],
    'Frequency': ['mean', 'median', 'min', 'max'],
    'Monetary': ['mean', 'median', 'min', 'max']
}).round(2)

print("Cluster Summary Statistics:")
print("=" * 80)
cluster_summary

In [ ]:
# Simplified cluster profile
cluster_profile = rfm.groupby('Cluster').agg({
    'Recency': 'mean',
    'Frequency': 'mean',
    'Monetary': 'mean'
}).round(2)

cluster_profile['Customer_Count'] = rfm.groupby('Cluster').size()
cluster_profile['Percentage'] = (cluster_profile['Customer_Count'] / len(rfm) * 100).round(2)

print("\nCluster Profiles:")
print("=" * 60)
cluster_profile

In [ ]:
# Visualize cluster characteristics
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Cluster distribution
cluster_counts = rfm['Cluster'].value_counts().sort_index()
colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4']
axes[0, 0].bar(cluster_counts.index, cluster_counts.values, color=colors, edgecolor='white')
axes[0, 0].set_xlabel('Cluster', fontsize=11)
axes[0, 0].set_ylabel('Number of Customers', fontsize=11)
axes[0, 0].set_title('Customer Distribution by Cluster', fontsize=13, fontweight='bold')
for i, v in enumerate(cluster_counts.values):
    axes[0, 0].text(i, v + 20, str(v), ha='center', fontweight='bold')

# Average RFM by cluster
cluster_means = rfm.groupby('Cluster')[['Recency', 'Frequency', 'Monetary']].mean()

# Recency by cluster
axes[0, 1].bar(cluster_means.index, cluster_means['Recency'], color=colors, edgecolor='white')
axes[0, 1].set_xlabel('Cluster', fontsize=11)
axes[0, 1].set_ylabel('Average Recency (Days)', fontsize=11)
axes[0, 1].set_title('Average Recency by Cluster', fontsize=13, fontweight='bold')

# Frequency by cluster
axes[1, 0].bar(cluster_means.index, cluster_means['Frequency'], color=colors, edgecolor='white')
axes[1, 0].set_xlabel('Cluster', fontsize=11)
axes[1, 0].set_ylabel('Average Frequency', fontsize=11)
axes[1, 0].set_title('Average Frequency by Cluster', fontsize=13, fontweight='bold')

# Monetary by cluster
axes[1, 1].bar(cluster_means.index, cluster_means['Monetary'], color=colors, edgecolor='white')
axes[1, 1].set_xlabel('Cluster', fontsize=11)
axes[1, 1].set_ylabel('Average Monetary ($)', fontsize=11)
axes[1, 1].set_title('Average Monetary Value by Cluster', fontsize=13, fontweight='bold')

plt.tight_layout()
plt.savefig('../images/cluster_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# 3D Scatter Plot of Clusters
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(111, projection='3d')

colors_scatter = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4']
cluster_labels = ['Cluster 0', 'Cluster 1', 'Cluster 2', 'Cluster 3']

for cluster in range(4):
    cluster_data = rfm[rfm['Cluster'] == cluster]
    ax.scatter(cluster_data['Recency'], 
               cluster_data['Frequency'], 
               cluster_data['Monetary'],
               c=colors_scatter[cluster], 
               label=cluster_labels[cluster],
               alpha=0.6, 
               s=30)

ax.set_xlabel('Recency (Days)', fontsize=10)
ax.set_ylabel('Frequency', fontsize=10)
ax.set_zlabel('Monetary ($)', fontsize=10)
ax.set_title('3D Visualization of Customer Segments', fontsize=14, fontweight='bold')
ax.legend(loc='upper left')

plt.tight_layout()
plt.savefig('../images/cluster_3d_visualization.png', dpi=150, bbox_inches='tight')
plt.show()

## 8. Predicting Cluster for New Customers

The trained model can be used to assign new customers to existing segments based on their RFM values.

In [ ]:
def predict_customer_segment(recency, frequency, monetary, scaler, kmeans_model):
    """
    Predict the cluster/segment for a new customer based on RFM values.
    
    Parameters:
    -----------
    recency : int
        Days since last purchase
    frequency : int
        Number of transactions
    monetary : float
        Total spending amount
    scaler : StandardScaler
        Fitted scaler object
    kmeans_model : KMeans
        Trained K-Means model
    
    Returns:
    --------
    int : Predicted cluster number
    """
    new_customer = np.array([[recency, frequency, monetary]])
    new_customer_scaled = scaler.transform(new_customer)
    cluster = kmeans_model.predict(new_customer_scaled)
    return cluster[0]

# Example: Predict segment for a new customer
new_recency = 30
new_frequency = 5
new_monetary = 3000

predicted_cluster = predict_customer_segment(
    new_recency, new_frequency, new_monetary, scaler, kmeans
)

print(f"New Customer Profile:")
print(f"  - Recency: {new_recency} days")
print(f"  - Frequency: {new_frequency} transactions")
print(f"  - Monetary: ${new_monetary}")
print(f"\nPredicted Segment: Cluster {predicted_cluster}")

## 9. Export Results

Save the segmented customer data for further analysis or business use.

In [ ]:
# Export segmented customers to CSV
output_path = '../data/customer_segments.csv'

# Reset index to include Customer ID as a column
rfm_export = rfm.reset_index()
rfm_export.to_csv(output_path, index=False)

print(f"Customer segments exported to: {output_path}")
print(f"Total customers segmented: {len(rfm_export)}")

## 10. Conclusion

### Summary
This project successfully implemented customer segmentation using K-Means clustering on the Online Retail II dataset. Key achievements include:

1. **Data Preprocessing:** Cleaned 541,910 records, resulting in 397,885 valid transactions from 4,338 unique customers.

2. **Feature Engineering:** Created RFM features that capture essential customer behavior patterns.

3. **Model Selection:** Used Elbow Method and Silhouette Score to determine optimal k=4 clusters.

4. **Customer Segments:** Identified 4 distinct customer groups with actionable business implications.

### Why No Train/Test Split?
Unlike supervised learning, unsupervised clustering doesn't require train/test splits because:
- There are no target labels to predict
- The goal is pattern discovery, not generalization to unseen labels
- Evaluation relies on internal metrics (WCSS, Silhouette Score) rather than prediction accuracy

### Business Recommendations
Based on cluster characteristics, businesses can implement targeted strategies:
- **High-Value Customers:** VIP programs, exclusive offers
- **At-Risk Customers:** Re-engagement campaigns
- **New Customers:** Onboarding sequences
- **Occasional Buyers:** Loyalty incentives

---
**For more details, see the [Project Report](../docs/PROJECT_REPORT.md) and [Methodology](../docs/METHODOLOGY.md).**